# 초반 세팅 부분(데이터 타입 최적화)

## 데이터 불러와서 미리 살펴본 데이터의 형태에 맞게 데이터 타입 변환
그리고 parquet파일로 저장

In [1]:
import os
from glob import glob

import pandas as pd
from tqdm.auto import tqdm

# directory = "../dataset/Looker Ecommerce BigQuery Dataset/*.csv"
def data_set_open_dictionary(directory):
    # 파일 데이터를 저장할 빈 사전 초기화
    file_dict = {}

    for file_path in tqdm(glob(directory)):
        standardized_path = os.path.normpath(file_path)
        file_name = standardized_path.split(os.sep)[-1].replace(".csv", "")
        # 파일명을 키로 하여 데이터프레임을 사전에 저장
        file_dict[file_name] = pd.read_csv(file_path)

    return file_dict

def data_set_display(file_dict):
    for df in tqdm(file_dict):
        print(df)
        file_dict[df].info()
        display(file_dict[df].head(3))

In [2]:
typechange_dict ={
    "distribution_centers": 
        {
        "int32_cols": [],
        "int16_cols": [],
        "int8_cols": ["id"],
        "float32_cols": ["latitude", "longitude"],
        "cate_cols": ["name"]
        }, 
    "events": 
        {
        "int32_cols": ["id","user_id"],
        "int16_cols": [],
        "int8_cols": ["sequence_number"],
        "float32_cols": [],
        "cate_cols": ["city", "state", "browser", "traffic_source", "event_type"]
        },
    "inventory_items": 
        {
        "int32_cols": ["id"],
        "int16_cols": ["product_id"],
        "int8_cols": ["product_distribution_center_id"],
        "float32_cols": [],
        "cate_cols": ["product_category", "product_brand", "product_department"]
        },
    "orders": 
        {
        "int32_cols": ["order_id", "user_id"],
        "int16_cols": [],
        "int8_cols": ["num_of_item"],
        "float32_cols": [],
        "cate_cols": ["status", "gender"]
        },
    "order_items": 
        {
        "int32_cols": ["id", "order_id", "user_id", "inventory_item_id"],
        "int16_cols": ["product_id"],
        "int8_cols": [],
        "float32_cols": ["sale_price"],
        "cate_cols": ["status"]
        },
    "products": 
        {
        "int32_cols": [],
        "int16_cols": ["id"],
        "int8_cols": ["distribution_center_id"],
        "float32_cols": ["cost", "retail_price"],
        "cate_cols": ["category", "brand", "department"]
        },
    "users": 
        {
        "int32_cols": ["id"],
        "int16_cols": [],
        "int8_cols": ["age"],
        "float32_cols": ["latitude", "longitude"],
        "cate_cols": ["first_name", "last_name", "gender", "state", "city", "country", "traffic_source"]
        },
    }
type_value = {"int32_cols": 2_147_483_647, "int16": 32_727, "int8_cols": 127, "float16_cols": 65_504, "cate_cols": 10_000}

In [5]:
directory = "../dataset/Looker Ecommerce BigQuery Dataset/*.csv"

file_dict = data_set_open_dictionary(directory)

  0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
for df_name, df in tqdm(file_dict.items()):
    memory_usage_before = df.memory_usage().sum()
    df. info()
    # datetime 바꾸기
    for col_datetime in df.columns[df.columns.str.endswith('_at')]:
        df[col_datetime] = pd.to_datetime(df[col_datetime], yearfirst=True, format="mixed", utc=True)
        
    df.info()
    for type_name, cols in typechange_dict[df_name].items():
        if type_name == "int32_cols":
            for col in cols:
                assert abs(df[col].max()) < 2_147_483_647 
                df[col] = df[col].astype(pd.Int32Dtype())
                # df.info()
        elif type_name == "int16_cols":
            for col in cols:
                assert abs(df[col].max()) < 2_147_483_647 
                df[col] = df[col].astype(pd.Int16Dtype())
                # df.info()
        elif type_name == "int8_cols":
            for col in cols:
                assert abs(df[col].max()) < 127
                df[col] = df[col].astype(pd.Int8Dtype())
                # df.info()
        elif type_name == "float32_cols": #"float16_cols": parquet는 16지원안한다는 것 같아서
            for col in cols:
                assert abs(df[col].max()) < 3.4e38
                df[col] = df[col].astype(pd.Float32Dtype())
                # df.info()
        elif type_name == "cate_cols":
            for col in cols:
                assert df[col].nunique() < 10_000
                df[col] = df[col].astype("category")
                # df.info()
    df.info()
    memory_usage_after = df.memory_usage().sum()
    reduction_ratio = 1 - (memory_usage_after / memory_usage_before)

    print(f"Memory Usage: {memory_usage_before:,} -> {memory_usage_after:,} ({reduction_ratio*100:.2f}% reduced)")
    
    # df.to_parquet(f"../dataset/Looker Ecommerce BigQuery Dataset/Optimization/{df_name}_optimized.parquet")
    # df.to_csv(f"../dataset/Looker Ecommerce BigQuery Dataset/Optimization/{df_name}_optimized.csv")

## 데이터 정보 확인해보기
데이터 종류 몇가지 있는지, 크기 몇부터 몇까지인지, 날짜 몇부터 몇까지 있는지

In [3]:
def type_check(df):
    for col, data_type in df.dtypes.items():
        if data_type in ("object", "category"):
            ser_target = df[col].value_counts()
            print(f"{col}({data_type}): {len(ser_target):,}가지")
        elif data_type in ("float", "int", "int64","Int8", "Int16", "Int32"):
            max_value = df[col].max()
            min_value = df[col].min()
            if (((df[col].dropna() % 1) != 0).sum() == 0) or (data_type == "int"):
                target_data_type = "int"
            else:
                target_data_type = "float"
            print(f"{col}({target_data_type}): {min_value:,.2f} ~ {max_value:,.2f}")
        elif data_type in ("datetime64[ns]", "datetime64[ns, UTC]"):
            max_date = df[col].max()
            min_date = df[col].min()
            print(f"{col}({data_type}): {min_date} ~ {max_date}")
        else:
            raise Exception("New data type:", data_type)

In [6]:
for i, df in file_dict.items():
    print(i)
    type_check(df)
    df.info()
    print("---------------------------")

distribution_centers
id(int): 1.00 ~ 10.00
name(object): 10가지
latitude(float): 29.76 ~ 41.84
longitude(float): -118.25 ~ -73.78
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         10 non-null     int64  
 1   name       10 non-null     object 
 2   latitude   10 non-null     float64
 3   longitude  10 non-null     float64
dtypes: float64(2), int64(1), object(1)
memory usage: 448.0+ bytes
---------------------------
inventory_items
id(int): 1.00 ~ 490,705.00
product_id(int): 1.00 ~ 29,120.00
created_at(object): 463,339가지
sold_at(object): 181,526가지
cost(float): 0.01 ~ 557.15
product_category(object): 26가지
product_name(object): 27,236가지
product_brand(object): 2,752가지
product_retail_price(float): 0.02 ~ 999.00
product_department(object): 2가지
product_sku(object): 29,046가지
product_distribution_center_id(int): 1.00 ~ 10.00
<class 'pandas.core.frame.D

# 1차 최적화 시킨 데이터 불러오기

In [1]:
import pandas as pd

distribution_centers = pd.read_parquet('../dataset/Looker Ecommerce BigQuery Dataset/Optimization/distribution_centers_optimized.parquet')
# events_file = pd.read_parquet('../dataset/Looker Ecommerce BigQuery Dataset/Optimization/events_optimized.parquet')
inventory_items = pd.read_parquet('../dataset/Looker Ecommerce BigQuery Dataset/Optimization/inventory_items_optimized.parquet')
orders = pd.read_parquet('../dataset/Looker Ecommerce BigQuery Dataset/Optimization/orders_optimized.parquet')
order_items = pd.read_parquet('../dataset/Looker Ecommerce BigQuery Dataset/Optimization/order_items_optimized.parquet')
products = pd.read_parquet('../dataset/Looker Ecommerce BigQuery Dataset/Optimization/products_optimized.parquet')
users = pd.read_parquet('../dataset/Looker Ecommerce BigQuery Dataset/Optimization/users_optimized.parquet')

In [2]:
events1 = pd.read_parquet("../dataset/Looker Ecommerce BigQuery Dataset/Optimization/events_optimized_divided1.parquet")
events2 = pd.read_parquet("../dataset/Looker Ecommerce BigQuery Dataset/Optimization/events_optimized_divided2.parquet")
events = pd.concat([events1, events2]).sort_index()

# 컬럼명 변경

- ~~distribution_centers~~
    - id -> distribution_center_id     
    - name -> distribution_centers_name     
    - 위도 경도 컬럼명 (users) 변경하는게 좋을 것 같음 고민중 서로 아마 다른 값들을 가지고 있을 것임 
- events
    - id -> event_id
    - created_at -> event_created_at
    - ~~city, state, postal_code (users) 같은 지 확인해봐야함 변경하는게 좋을 것 같기는 함~~ -> 같음
    - traffic_source -> event_traffic_source
- inventory_items
    - id -> inventory_item_id
    - created_at -> inventory_item_created_at
    - sold_at -> 고민중 겹치는 것은 없는데 통일해주는게 쓰기 편하지 않을까 생각이 들어서, 주문에 있는 created_at과 같을 가능성 높음
    - ~~cost (products)고민중 중복되기는 한데 둘이 같은 것을 의미해서~~
    - ~~product_category, product_name, product_brand, product_retail_price, product_department, product_sku, product_distribution_center_id~~
    고민중 이름 다르게 되어있는데 같은 것이라서 하지만 달라서 문제되는 것은 없긴 해서 통일시켜줄까 생각중이지만 그냥 두는게 나을듯
- ~~orders~~
    - status(order_items), gender(users) 고민중 이름 중복은 되는데 둘이 같은 것을 의미해서 그냥 냅두고 안쓸까 생각중
    - created_at, returned_at, shipped_at, delivered_at order_items의 것과 둘이 같은 것인지 확인해봐야함 -> created는 좀 다르긴 한데 굳이 이 테이블 자체를 잘 안쓸듯함
- order_items
    - id -> order_item_id
    - ~~status(order) 고민중 이름 중복은 되는데 둘이 같은 것을 의미해서 그냥 냅두고 잘 합칠 생각 중~~
    - ~~created_at, returned_at, shipped_at, delivered_at orders의 것과 둘이 같은 것인지 확인해봐야함~~ -> 그냥 둠
- ~~products~~
    - id -> product_id
    - ~~cost (inventory_items)고민중 중복되기는 한데 둘이 같은 것을 의미해서~~
    - name -> product_name
    - ~~distribution_center_id 이라고 이름 같게 해줄 예정이라 합칠려고 고려중~~
- users
    - id -> user_id (다른 곳에 일치하는 컬럼 많음 어떻게 해줄지 고민중)
    - ~~gender (orders)와 겹쳐서 그런데 orders 테이블을 안쓰게될듯?~~
    - ~~state, postal_code, city (events) 같은 지 확인해봐야함 변경하는게 좋을 것 같기는 함~~
    - 위도 경도에 대해서 (distribution_centers) 컬럼명 변경하는게 좋을 것 같음 고민중 서로 아마 다른 값들을 가지고 있을 것임
    - traffic_source -> user_traffic_source
    - created_at -> user_created_at

## 확인 과정

### 위도 경도 확인 -> 확인 결과 예상대로 의미가 다르다보니 아예 다른 좌표들

In [4]:
distribution_centers

,id,name,latitude,longitude
0,1,Memphis TN,35.1174,-89.9711
1,2,Chicago IL,41.8369,-87.6847
2,3,Houston TX,29.7604,-95.3698
3,4,Los Angeles CA,34.0500,-118.2500
4,5,New Orleans LA,29.9500,-90.0667
5,6,Port Authority of New York/New Jersey NY/NJ,40.6340,-73.7834
6,7,Philadelphia PA,39.9500,-75.1667
7,8,Mobile AL,30.6944,-88.0431
8,9,Charleston SC,32.7833,-79.9333
9,10,Savannah GA,32.0167,-81.1167


In [8]:
print(distribution_centers['latitude'].values)
print(distribution_centers['longitude'].values)

[35.1174 41.8369 29.7604 34.05   29.95   40.634  39.95   30.6944 32.7833
 32.0167]
[ -89.9711  -87.6847  -95.3698 -118.25    -90.0667  -73.7834  -75.1667
  -88.0431  -79.9333  -81.1167]


In [19]:
35.1174 in distribution_centers['latitude'].values
# 35.1174 in users['latitude'].values

True

In [18]:
# 각각 하나씩 돌려서 있는지 확인해볼 예정
count_lat = 0
for lat in distribution_centers['longitude'].values:
    if lat in users['longitude'].values:
        print("있음")
        count_lat += 1
        
print(count_lat)

count_long = 0
for lat in distribution_centers['latitude'].values:
    if lat in users['latitude'].values:
        print("있음")
        count_long += 1
        
print(count_long)

0
0


결론으로 값이 같은 게 없고 의미상으로도 다르니 변수명 다르게 변경

### events와 users의 city, state, postal_code 같은지 확인 -> 확인결과 같음

In [3]:
events.dropna()[['user_id', 'city' , 'state', 'postal_code']]

,user_id,city,state,postal_code
1251,35092,Sapporo,Hokkaido,003-0030
1252,34028,Sapporo,Hokkaido,007-0890
1253,13199,São Paulo,São Paulo,02220-000
1257,87818,São Paulo,São Paulo,02675-031
1259,46234,São Paulo,São Paulo,02675-031
...,...,...,...,...
2431958,62906,St Helens,England,WA9
2431959,3333,Orrell,England,WN5
2431960,42386,Orrell,England,WN5
2431961,77546,Wolverhampton,England,WV6


In [4]:
compare_address_df = events.dropna()[['user_id', 'city' , 'state', 'postal_code']].merge(users[['id', 'city', 'state', 'postal_code']], left_on='user_id', right_on='id')

In [5]:
compare_address_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1294116 entries, 0 to 1294115
Data columns (total 8 columns):
 #   Column         Non-Null Count    Dtype   
---  ------         --------------    -----   
 0   user_id        1294116 non-null  Int32   
 1   city_x         1294116 non-null  category
 2   state_x        1294116 non-null  category
 3   postal_code_x  1294116 non-null  object  
 4   id             1294116 non-null  Int32   
 5   city_y         1294116 non-null  category
 6   state_y        1294116 non-null  category
 7   postal_code_y  1294116 non-null  object  
dtypes: Int32(2), category(4), object(2)
memory usage: 42.6+ MB


카테고리의 비교의 경우에는 둘이 둘다 카테고리더라도 같은 카테고리 타입이여야하기 때문에 일단 둘의 카테고리 값들 목록이 같은지 확인하고  
둘 중 하나의 카테고리로 통일 시켜준 다음에 비교해야하는듯

In [8]:
print(set(compare_address_df['city_x'].cat.categories) != set(compare_address_df['city_y'].cat.categories))
compare_address_df['city_y'] = compare_address_df['city_y'].astype(compare_address_df['city_x'].dtype)
(compare_address_df['city_x']!=compare_address_df['city_y']).sum()

True


In [12]:
print(set(compare_address_df['state_x'].cat.categories) != set(compare_address_df['state_y'].cat.categories))
compare_address_df['state_y'] = compare_address_df['state_y'].astype(compare_address_df['state_x'].dtype)
(compare_address_df['state_x']!=compare_address_df['state_y']).sum()

True


0

In [14]:
(compare_address_df['postal_code_x']!=compare_address_df['postal_code_y']).sum()

0

### order와 order_items created_at, returned_at, shipped_at, delivered_at 같은지 확인
returned_at, shipped_at, delivered_at은 null값 아니면 같은 것 확인됨 null일 경우 null로 같은 듯 하지만 다르다고 표시됨
created_at는 정확히 어떤 규칙으로 orders의 created_at이 정해진지 모르겠음 하지만 orders 굳이 안써도 될듯해서 그냥 안쓰는 쪽으로 결정

In [17]:
orders['order_id'].is_monotonic_increasing # 이 컬럼이 속성은 컬럼 값이 단조롭게 증가하는지(즉, 정렬되어 있는지) 여부

False

In [47]:
# df1과 df2 병합
merged_df = orders.merge(order_items, on=['user_id', 'order_id'])
# 'a' 컬럼과 'b' 컬럼의 값이 같은 행만 필터링
matched_rows_created = merged_df[merged_df['created_at_x'] == merged_df['created_at_y']].reset_index()
mismatched_rows_created = merged_df[merged_df['created_at_x'] != merged_df['created_at_y']].reset_index()
matched_rows_shipped = merged_df[merged_df['shipped_at_x'] == merged_df['shipped_at_y']].reset_index()
mismatched_rows_shipped = merged_df[merged_df['shipped_at_x'] != merged_df['shipped_at_y']].reset_index()
matched_rows_delivered = merged_df[merged_df['delivered_at_x'] == merged_df['delivered_at_y']].reset_index()
mismatched_rows_delivered = merged_df[merged_df['delivered_at_x'] != merged_df['delivered_at_y']].reset_index()
matched_rows_returned = merged_df[merged_df['returned_at_x'] == merged_df['returned_at_y']].reset_index()
mismatched_rows_returned = merged_df[merged_df['returned_at_x'] != merged_df['returned_at_y']].reset_index()

In [83]:
print("created")
print(len(matched_rows_created))
print(len(mismatched_rows_created))
print("----------------")
print("shipped")
print(len(matched_rows_shipped))
print(len(mismatched_rows_shipped))
print("----------------")
print("delivered")
print(len(matched_rows_delivered))
print(len(mismatched_rows_delivered))
print("----------------")
print("returned")
print(len(matched_rows_returned))
print(len(mismatched_rows_returned))

created
11
181748
----------------
shipped
118281
63478
----------------
delivered
63841
117918
----------------
returned
18232
163527


In [51]:
matched_rows_created[['order_id','user_id', 'status_x', 'status_y', 'created_at_x', 'created_at_y', 'num_of_item']]

,order_id,user_id,status_x,status_y,created_at_x,created_at_y,num_of_item
0,41168,32940,Complete,Complete,2023-09-14 08:56:00+00:00,2023-09-14 08:56:00+00:00,4
1,55407,44381,Returned,Returned,2020-08-06 15:13:00+00:00,2020-08-06 15:13:00+00:00,2
2,925,744,Shipped,Shipped,2021-05-28 06:41:00+00:00,2021-05-28 06:41:00+00:00,1
3,47245,37793,Shipped,Shipped,2023-05-26 13:35:00+00:00,2023-05-26 13:35:00+00:00,1
4,112098,89542,Complete,Complete,2023-08-25 05:10:00+00:00,2023-08-25 05:10:00+00:00,4
5,13269,10649,Processing,Processing,2024-01-03 10:45:00+00:00,2024-01-03 10:45:00+00:00,1
6,8297,6624,Shipped,Shipped,2023-12-15 08:02:00+00:00,2023-12-15 08:02:00+00:00,2
7,8393,6699,Shipped,Shipped,2023-05-09 17:00:00+00:00,2023-05-09 17:00:00+00:00,1
8,30673,24530,Shipped,Shipped,2023-04-30 00:57:00+00:00,2023-04-30 00:57:00+00:00,1
9,73503,58690,Shipped,Shipped,2021-12-05 11:10:00+00:00,2021-12-05 11:10:00+00:00,1


In [70]:
user_number = 78588
display(mismatched_rows_created[mismatched_rows_created['user_id']==user_number])#[['user_id', 'status_x', 'status_y', 'created_at_x', 'created_at_y']]
display(orders[orders['user_id']==user_number])
display(order_items[order_items['user_id']==user_number])

,index,order_id,user_id,status_x,gender,created_at_x,returned_at_x,shipped_at_x,delivered_at_x,num_of_item,id,product_id,inventory_item_id,status_y,created_at_y,shipped_at_y,delivered_at_y,returned_at_y,sale_price
122670,122674,98483,78588,Complete,M,2023-07-17 06:39:00+00:00,NaT,2023-07-17 08:36:00+00:00,2023-07-17 13:03:00+00:00,1,143048,27448,386164,Complete,2023-07-17 04:24:44+00:00,2023-07-17 08:36:00+00:00,2023-07-17 13:03:00+00:00,NaT,49.990002


,order_id,user_id,status,gender,created_at,returned_at,shipped_at,delivered_at,num_of_item
84472,98483,78588,Complete,M,2023-07-17 06:39:00+00:00,NaT,2023-07-17 08:36:00+00:00,2023-07-17 13:03:00+00:00,1
121224,98482,78588,Shipped,M,2022-07-06 06:39:00+00:00,NaT,2022-07-09 03:33:00+00:00,NaT,1


,id,order_id,user_id,product_id,inventory_item_id,status,created_at,shipped_at,delivered_at,returned_at,sale_price
45575,143047,98482,78588,20156,386160,Shipped,2022-07-06 06:39:00+00:00,2022-07-09 03:33:00+00:00,NaT,NaT,24.750000
110219,143048,98483,78588,27448,386164,Complete,2023-07-17 04:24:44+00:00,2023-07-17 08:36:00+00:00,2023-07-17 13:03:00+00:00,NaT,49.990002


In [79]:
matched_rows_returned[~matched_rows_returned['order_id'].isin([41168, 55407, 925, 47245, 112098, 13269, 8297, 8393, 30673, 73503, 98482])][['order_id','user_id', 'status_x', 'status_y', 'returned_at_x', 'returned_at_y', 'num_of_item']]

,order_id,user_id,status_x,status_y,returned_at_x,returned_at_y,num_of_item
0,14,11,Returned,Returned,2024-01-16 21:15:00+00:00,2024-01-16 21:15:00+00:00,1
1,19,12,Returned,Returned,2023-07-13 13:41:00+00:00,2023-07-13 13:41:00+00:00,1
2,29,19,Returned,Returned,2022-08-01 21:04:00+00:00,2022-08-01 21:04:00+00:00,1
3,34,24,Returned,Returned,2023-02-13 15:36:00+00:00,2023-02-13 15:36:00+00:00,1
4,96,73,Returned,Returned,2022-11-27 00:11:00+00:00,2022-11-27 00:11:00+00:00,2
...,...,...,...,...,...,...,...
18227,125123,99907,Returned,Returned,2023-09-19 16:28:00+00:00,2023-09-19 16:28:00+00:00,1
18228,125129,99914,Returned,Returned,2023-03-12 15:47:00+00:00,2023-03-12 15:47:00+00:00,1
18229,125143,99925,Returned,Returned,2022-11-11 19:13:00+00:00,2022-11-11 19:13:00+00:00,1
18230,125185,99965,Returned,Returned,2022-01-19 12:10:00+00:00,2022-01-19 12:10:00+00:00,2


#### 오래걸려서 폐기한 방법

In [27]:
from tqdm.auto import tqdm

match_list = []
mismatch_list = []
for index, row in tqdm(orders.iterrows()):
    # df2에서 일치하는 id2 찾기
    matching_row = order_items[order_items['order_id'] == row['order_id']]
    if not matching_row.empty:
        # 'a' 컬럼과 'b' 컬럼의 값 비교
        if row['created_at'] == matching_row['created_at'].values[0]:
            match_list.append(index)
        else:
            mismatch_list.append(index)

0it [00:00, ?it/s]

In [28]:
print(len(match_list))
print(len(mismatch_list))

0
125226


In [21]:
orders.sort_values('order_id').reset_index(drop=True)

,order_id,user_id,status,gender,created_at,returned_at,shipped_at,delivered_at,num_of_item
0,1,1,Shipped,F,2022-07-18 12:55:00+00:00,NaT,2022-07-20 16:42:00+00:00,NaT,3
1,2,2,Complete,M,2022-02-20 10:34:00+00:00,NaT,2022-02-22 12:23:00+00:00,2022-02-23 18:29:00+00:00,1
2,3,3,Processing,M,2023-04-23 09:13:00+00:00,NaT,NaT,NaT,1
3,4,3,Processing,M,2023-08-08 09:13:00+00:00,NaT,NaT,NaT,1
4,5,3,Shipped,M,2023-03-10 09:13:00+00:00,NaT,2023-03-11 12:33:00+00:00,NaT,1
...,...,...,...,...,...,...,...,...,...
125221,125222,99998,Processing,M,2023-01-28 07:27:00+00:00,NaT,NaT,NaT,1
125222,125223,99998,Processing,M,2023-07-22 07:27:00+00:00,NaT,NaT,NaT,1
125223,125224,99999,Shipped,M,2023-12-22 11:29:00+00:00,NaT,2023-12-24 00:44:00+00:00,NaT,2
125224,125225,100000,Returned,F,2021-08-21 03:04:00+00:00,2021-08-26 03:21:00+00:00,2021-08-22 16:39:00+00:00,2021-08-25 18:48:00+00:00,1


## 컬럼명 변경

In [7]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 15 columns):
 #   Column          Non-Null Count   Dtype              
---  ------          --------------   -----              
 0   id              100000 non-null  Int32              
 1   first_name      100000 non-null  category           
 2   last_name       100000 non-null  category           
 3   email           100000 non-null  object             
 4   age             100000 non-null  Int8               
 5   gender          100000 non-null  category           
 6   state           100000 non-null  category           
 7   street_address  100000 non-null  object             
 8   postal_code     100000 non-null  object             
 9   city            99042 non-null   category           
 10  country         100000 non-null  category           
 11  latitude        100000 non-null  float64            
 12  longitude       100000 non-null  float64            
 13  traffic_source 

In [37]:
distribution_centers = distribution_centers.rename(
    columns={
        'distribution_centers_id': 'distribution_center_id', 
        'distribution_centers_name': 'distribution_center_name', 
        'latitude': 'distribution_center_latitude', 
        'longitude': 'distribution_center_longitude'
        }
    )

In [20]:
distribution_centers = distribution_centers.rename(
    columns={
        'id': 'distribution_center_id', 
        'name': 'distribution_center_name', 
        'latitude': 'distribution_center_latitude', 
        'longitude': 'distribution_center_longitude'
        }
    )
inventory_items = inventory_items.rename(
    columns={
        'id': 'inventory_item_id', 
        'created_at': 'inventory_item_created_at', 
        'sold_at': 'inventory_item_sold_at', 
        }
    )
# orders = orders.rename(columns={})
order_items = order_items.rename(columns={'id': 'order_item_id'})
products = products.rename(columns={'id': 'product_id', 'name': 'product_name'})
users = users.rename(
    columns={
        'id': 'user_id', 
        'latitude': 'user_latitude', 
        'longitude': 'user_longitude', 
        'traffic_source': 'user_traffic_source', 
        'created_at': 'user_created_at'
        }
    )
events1 = events1.rename(
    columns={
        'id': 'event_id', 
        'created_at': 'event_created_at', 
        'traffic_source': 'event_traffic_source', 
        }
    )
events2 = events2.rename(
    columns={
        'id': 'event_id', 
        'created_at': 'event_created_at', 
        'traffic_source': 'event_traffic_source', 
        }
    )

In [3]:
display(distribution_centers.head(1))
display(inventory_items.head(1))
display(orders.head(1))
display(order_items.head(1))
display(products.head(1))
display(users.head(1))
display(events1.head(1))
display(events2.head(1))

,distribution_center_id,distribution_center_name,distribution_center_latitude,distribution_center_longitude
0,1,Memphis TN,35.1174,-89.9711


,inventory_item_id,product_id,inventory_item_created_at,inventory_item_sold_at,cost,product_category,product_name,product_brand,product_retail_price,product_department,product_sku,product_distribution_center_id
0,67971,13844,2022-07-02 07:09:20+00:00,2022-07-24 06:33:20+00:00,2.76804,Accessories,(ONE) 1 Satin Headband,Funny Girl Designs,6.99,Women,2A3E953A5E3D81E67945BCE5519F84C8,7


,order_id,user_id,status,gender,created_at,returned_at,shipped_at,delivered_at,num_of_item
0,8,5,Cancelled,F,2022-10-20 10:03:00+00:00,NaT,NaT,NaT,3


,order_item_id,order_id,user_id,product_id,inventory_item_id,status,created_at,shipped_at,delivered_at,returned_at,sale_price
0,152013,104663,83582,14235,410368,Cancelled,2023-05-07 06:08:40+00:00,NaT,NaT,NaT,0.02


,product_id,cost,category,product_name,brand,retail_price,department,sku,distribution_center_id
0,13842,2.51875,Accessories,Low Profile Dyed Cotton Twill Cap - Navy W39S55D,MG,6.25,Women,EBD58B8A3F1D72F4206201DA62FB1204,1


,user_id,first_name,last_name,email,age,gender,state,street_address,postal_code,city,country,user_latitude,user_longitude,user_traffic_source,user_created_at
0,457,Timothy,Bush,timothybush@example.net,65,M,Acre,87620 Johnson Hills,69917-400,Rio Branco,Brasil,-9.945568,-67.83561,Search,2022-07-19 13:51:00+00:00


,event_id,user_id,sequence_number,session_id,event_created_at,ip_address,city,state,postal_code,browser,event_traffic_source,uri,event_type
3,2250597,<NA>,3,96f1d44e-9621-463c-954c-d8deb7fffe7f,2022-03-30 10:56:00+00:00,137.25.222.160,Cajamar,São Paulo,07750-000,Chrome,Adwords,/cancel,cancel


,event_id,user_id,sequence_number,session_id,event_created_at,ip_address,city,state,postal_code,browser,event_traffic_source,uri,event_type
0,2198523,<NA>,3,83889ed2-2adc-4b9a-af5d-154f6998e778,2021-06-17 17:30:00+00:00,138.143.9.202,São Paulo,São Paulo,02675-031,Chrome,Adwords,/cancel,cancel


In [4]:
distribution_centers.info()
inventory_items.info()
orders.info()
order_items.info()
products.info()
users.info()
events1.info()
events2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, 0 to 9
Data columns (total 4 columns):
 #   Column                         Non-Null Count  Dtype   
---  ------                         --------------  -----   
 0   distribution_center_id         10 non-null     Int8    
 1   distribution_center_name       10 non-null     category
 2   distribution_center_latitude   10 non-null     float64 
 3   distribution_center_longitude  10 non-null     float64 
dtypes: Int8(1), category(1), float64(2)
memory usage: 650.0 bytes
<class 'pandas.core.frame.DataFrame'>
Index: 490705 entries, 0 to 490704
Data columns (total 12 columns):
 #   Column                          Non-Null Count   Dtype              
---  ------                          --------------   -----              
 0   inventory_item_id               490705 non-null  Int32              
 1   product_id                      490705 non-null  Int16              
 2   inventory_item_created_at       490705 non-null  datetime64[ns, 

In [24]:
print(distribution_centers.columns[distribution_centers.columns.duplicated()])

Index(['distribution_centers'], dtype='object')


In [40]:
# df.to_parquet(f"../dataset/Looker Ecommerce BigQuery Dataset/Optimization/{df_name}_optimized.parquet")
distribution_centers.to_parquet('../dataset/Looker Ecommerce BigQuery Dataset/Optimization/distribution_centers_optimized.parquet')
inventory_items.to_parquet('../dataset/Looker Ecommerce BigQuery Dataset/Optimization/inventory_items_optimized.parquet')
orders.to_parquet('../dataset/Looker Ecommerce BigQuery Dataset/Optimization/orders_optimized.parquet')
order_items.to_parquet('../dataset/Looker Ecommerce BigQuery Dataset/Optimization/order_items_optimized.parquet')
products.to_parquet('../dataset/Looker Ecommerce BigQuery Dataset/Optimization/products_optimized.parquet')
users.to_parquet('../dataset/Looker Ecommerce BigQuery Dataset/Optimization/users_optimized.parquet')
events1.to_parquet("../dataset/Looker Ecommerce BigQuery Dataset/Optimization/events_optimized_divided1.parquet")
events2.to_parquet("../dataset/Looker Ecommerce BigQuery Dataset/Optimization/events_optimized_divided2.parquet")

# 변수명 결정